In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.planning.filtering.latticized_filter import SCENE_PATH
[fname for fname in os.listdir(SCENE_PATH) if "ERROR" in fname]

['00000438-ERROR.pkl', '00000439-ERROR.pkl']

In [3]:
from pkg.controller.robot_config import RobotType
from pkg.utils.test_scripts import *

In [4]:
ROBOT_TYPE = RobotType.indy7gripper
VISUALIZE = True

In [5]:
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [6]:
from pkg.planning.filtering.latticized_filter import SCENE_PATH, ARM_SHAPE, GRASP_SHAPE, \
    WDH_GRASP, L_CELL_GRASP, OFFSET_ZERO_GRASP, WDH_ARM, L_CELL_ARM, OFFSET_ZERO_ARM
from pkg.planning.filtering.lattice_model.latticizer_py import get_centers

centers_grasp = get_centers(GRASP_SHAPE, L_CELL_GRASP, OFFSET_ZERO_GRASP)
centers_arm = get_centers(ARM_SHAPE, L_CELL_ARM, OFFSET_ZERO_ARM)


assert len(pscene.robot_chain_dict.values())==1, "not single robot scene"
shoulder_height = get_tf(
    gscene.urdf_content.joint_map[pscene.robot_chain_dict.values()[0]['joint_names'][1]].child, 
    crob.home_dict, pscene.gscene.urdf_content)[2, 3]

In [7]:
scene_data = load_pickle(os.path.join(SCENE_PATH, "00000439-ERROR.pkl"))
global_log = scene_data["global_log"]
arm_tar_idx = scene_data["arm_tar_idx"]
grasp_tool_idx = scene_data["grasp_tool_idx"]
grasp_tar_idx = scene_data["grasp_tar_idx"]
grasp_obj_idx = scene_data["grasp_obj_idx"]
radius, theta, height = scene_data["rth"]
gtem_args = scene_data["gtem_args"]
error_state = scene_data["error_state"]
offset_ee = cyl2cart(radius, theta, height)

load_gtem_args(gscene, gtem_args)

In [8]:
arm_tar_idx =  np.unravel_index(arm_tar_idx, shape=GRASP_SHAPE)
grasp_tool_idx = np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)
grasp_tar_idx = np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)
grasp_obj_idx = np.unravel_index(np.array(grasp_obj_idx).astype(np.int), shape=GRASP_SHAPE)

In [9]:
radius, theta, height = scene_data["rth"]

In [10]:
for i_c, center in enumerate(centers_arm[arm_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
    gscene.create_safe(GEOTYPE.SPHERE, "arm_%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0.5, 0.5, 0.5, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_tool_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tool_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)

for i_c, center in enumerate(centers_grasp[grasp_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tar_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 0, 0, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_obj_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "obj_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)

In [12]:
global_log.keys()

['from_state', 'to_state', 'actor', 'subject']

In [16]:
from_state = global_log["from_state"]
to_state = global_log["to_state"]
subject = global_log["subject"]
actor = global_log["actor"]
print("{} - {}".format(actor, subject))

grip0 - obj_0


In [18]:
from_state.binding_state

(('obj_0', 'bottom_p', 'wp', 'wp'),
 ('obj_1', 'bottom_p', 'wp', 'wp'),
 ('obj_2', 'bottom_p', 'wp', 'wp'))

In [19]:
to_state.binding_state

(('obj_0', 'bottom_p', 'wp', 'wp'),
 ('obj_1', 'bottom_p', 'wp', 'wp'),
 ('obj_2', 'bottom_p', 'wp', 'wp'))

In [20]:
from_state.Q

array([0., 0., 0., 0., 0., 0.])

In [21]:
to_state.Q

array([ 1.85235231,  1.06607314,  1.49469566, -1.56395496, -1.50639244,
        0.58163549])

In [23]:
gscene.show_pose(to_state.Q)

In [25]:
gscene.NAME_DICT["obj_0"].color = (1,0,1,0.8)

In [26]:
gtem = gscene.NAME_DICT["obj_0"]